In [1]:
import ee
import numpy as np
import pandas as pd
import pickle
import datetime

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWj-i6TNNxm6OuDjyaeutkVrWmPwrkbWXi6CZhE7z-pYN-WqWnAi4Jk



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [52]:
# sampling paramaters
randomSeed = 200
collectionLandsatL1c = "LANDSAT/LC08/C02/T1_TOA" # landsat image collection
collectionLandsatL2a= "LANDSAT/LC08/C02/T1_L2" # landsat image collection
cloudCoverL0X = 10 # maximum cloud cover of sampled Landsat images
maxSample = 10000 # maximum samples to extract from each Landsat image
outputScale = 500 # scale of output comparisons , increasing this will eventually exceed memory
subimageSize = 20000 # m dimension of subimage we will randomly extract from Landsat image to avoid memory limits
minfracAreaLC = 0.5 # minimum cloud free valid  Landsat subimage fraction
minfracAreaMCD15 = 0.5 # minimum cloud free valid MODIS area in Landsat subimage fraction's unmasked area
minfracAreaLCtoMCD = 0.25 # valid aggregated landsat fraction
minfracValidLC = 0.9 # minimum facation of valid landsat pixels in each aggregated modis pixels to estimate mean values
deltaRow = 10  # skip this many WRS rows when sampling Landsat scenes
deltaPath = 10 # skip this many WRS paths when sampling Landsat scenes
deltaDay = 15 # search this many contiguous days when looking for a cloud free Landsat scene
maxPixels = 300 # max landsat pixels allowed to be aggregated in an output pixel, should be about 20% larger than the ratio of (outputScale/30) (cannot exceed 1024)

#output files
unsatInputFileNameB36 = '/home/richard_fernandes/unsatLUTinputLC0XB36run2020'
unsatInputFileNameB45 = '/home/richard_fernandes/unsatLUTinputLC0XB45run2020'
unsatInputFileNameLat = '/home/richard_fernandes/unsatLUTinputLC0XLatrun2020'
unsatInputFileNameLon = '/home/richard_fernandes/unsatLUTinputLC0XLonrun2020'

unsatOutputFileName = '/home/richard_fernandes/unsatLUToutputLC0Xrun2020'
unsatYearFileName = '/home/richard_fernandes/unsatYearLC0Xrun2020'
unsatMonthFileName = '/home/richard_fernandes/unsatMonthLC0Xrun2020'
unsatDayFileName = '/home/richard_fernandes/unsatDayLC0Xrun2020'
unsatPathFileName = '/home/richard_fernandes/unsatPathLC0Xrun2020'
unsatRowFileName = '/home/richard_fernandes/unsatRowLC0Xrun2020'

satInputFileNameB36 = '/home/richard_fernandes/satLUTinputLC0XB36run2020'
satInputFileNameB45 = '/home/richard_fernandes/satLUTinputLC0XB45run2020'
satInputFileNameLat = '/home/richard_fernandes/satLUTinputLC0XLatrun2020'
satInputFileNameLon = '/home/richard_fernandes/satLUTinputLC0XLonrun2020'

satOutputFileName = '/home/richard_fernandes/satLUToutputLC0Xrun2020'
satYearFileName = '/home/richard_fernandes/satYearLC0Xrun2020'
satMonthFileName = '/home/richard_fernandes/satMonthLC0Xrun2020'
satDayFileName = '/home/richard_fernandes/satDayLC0Xrun2020'
satPathFileName = '/home/richard_fernandes/satPathLC0Xrun2020'
satRowFileName = '/home/richard_fernandes/satRowLC0Xrun2020'



In [50]:
#function to compute signed distance from images of pixel coordinates to a line defined by a list to two points
def getDist(image,pts) :
  
    # cast inputs
    image = ee.Image(image)
    pts = ee.List(pts)
    
    # identify coord layers
    x0 = image.select('x')
    y0 = image.select('y')
    
    # identify end points
    x1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(0)))
    x2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(0)))
    y1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(1)))
    y2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(1)))
    denom = ((x2.subtract(x1)).pow(ee.Image.constant(2)).add((y2.subtract(y1)).pow(ee.Image.constant(2)))).pow(ee.Image.constant(0.5))
    
    return ((((x2.subtract(x1)).multiply(y1.subtract(y0))).subtract((x1.subtract(x0)).multiply(y2.subtract(y1))))).divide(denom)
    



In [6]:
#  Estimate view zenith angle in a Landsat image based on lateral distance from satellite track
#  Assume flat earth and linear track within image
# deprecated as the L1C has vza
def getVZALC(imageLC) :

    # get point farthest from centroid p1
    points = ee.List(((imageLC.geometry().coordinates()).get(0)))
    dists0 = points.map( lambda point: ee.Geometry.Point(point).distance(imageLC.geometry().centroid()).multiply(-1))
    p1 = ee.List(points.sort(dists0).get(0))
    #print(p1)

    # get point farthest from p1:  p2
    dists1 = points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p1)).multiply(-1))
    p2 = ee.List(points.sort(dists1).get(0))
    #print(p2)

    # get point farthest from p1 and p2: p3
    dists2 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p2)).multiply(-1))
    dists2 =  ee.List.sequence(0,dists1.length().subtract(1),1).map(lambda index: ee.Number(dists1.get(index)).add(ee.Number(dists2.get(index))) )
    p3 = ee.List(points.sort(dists2).get(0))
    #p.ipynb_checkpoints/int(p3)

    # get point farthest from p3: p4
    dists3 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p3)).multiply(-1))
    p4 = ee.List(points.sort(dists3).get(0))
    #print(p4)

    # separate easting and northing coords of points
    ptsE = [p1.get(0) , p2.get(0), p3.get(0), p4.get(0)]
    ptsN = [p1.get(1) , p2.get(1), p3.get(1), p4.get(1)]
    #print(ptsE)
    #print(ptsN)

    # make list of corner points
    pts = [p1,p2,p3,p4]
    #print(pts)

    # separate East and West side corner points
    projectionLC =imageLC.select('QA_PIXEL').projection()
    ptWest = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(0)).transform(projectionLC).coordinates()
    ptNorth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(3)).transform(projectionLC).coordinates()
    ptSouth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(0)).transform(projectionLC).coordinates()
    ptEast = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(3)).transform(projectionLC).coordinates()

    ptsWest = ee.List([ptWest, ptNorth])
    ptsEast = ee.List([ptEast ,ptSouth])
    #print(ptsEast)
    #print(ptsWest )
    
    # compute distance between each pixel and line defined by East and West side corner points 
    imageLCCoords = ee.Image.pixelCoordinates(projectionLC).clip(imageLC.geometry())
    distEast = getDist(imageLCCoords ,ptsEast).abs()
    distWest = getDist(imageLCCoords ,ptsWest).abs()

    # compute VZA assuming +/-7.5degree scan angle 
    return (distEast.divide(distEast.add(distWest)).subtract(ee.Image.constant(0.5)).multiply(ee.Image.constant(7.5)))

In [7]:
# deprecated functions but I leave it here as it is useful

# clip the top and bottom rows of Landsat image  by specified degrees
def clipImageRows(image,clipDegrees):
    image = ee.Image(image);
    clipDegrees = ee.Number(clipDegrees);
    
    # get projection from first band
    projectionImage = image.select(image.BandNames().get(0)).projection()
    
    # get the bounding box of the image and parse co-ords
    imageBBOX = ee.List(image.geometry().bounds().coordinates().get(0));
    long1 = ee.Number(ee.List(imageBBOX.get(0)).get(0))
    long2 = ee.Number(ee.List(imageBBOX.get(1)).get(0))
    long3 = ee.Number(ee.List(imageBBOX.get(2)).get(0))
    long4 = ee.Number(ee.List(imageBBOX.get(3)).get(0))
    long5 = ee.Number(ee.List(imageBBOX.get(4)).get(0))
    lat1 = ee.Number(ee.List(imageBBOX.get(0)).get(1))
    lat2 = ee.Number(ee.List(imageBBOX.get(1)).get(1))
    lat3 = ee.Number(ee.List(imageBBOX.get(2)).get(1))
    lat4 = ee.Number(ee.List(imageBBOX.get(3)).get(1))
    lat5 = ee.Number(ee.List(imageBBOX.get(4)).get(1))

    # Make a smaller bounding box 
    return image.clip(ee.Geometry.Polygon( 
                              coords =  [ \
                                          ee.Geometry.LinearRing( \
                                            [ \
                                              [long1, lat1.add(clipDegrees)], \
                                              [long2, lat2.add(clipDegrees)], \
                                              [long3, lat3.subtract(clipDegrees)], \
                                              [long4, lat4.subtract(clipDegrees)], \
                                              [long5, lat5.add(clipDegrees)] \
                                            ]
                                          )
                                        ],
                                proj =  projectionImage ,
                                geodesic =  False,
                                maxError = 0.1));


def numberOfPixels(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width ).multiply( ee.Number( height ) )

def numberOfPixelsW(img) :
    imgDescription = ee.Algorithms.Describe( img )
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width )

def numberOfPixelsH(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    return  ee.Number(height )

In [57]:
# coded LUT for output MODIS MCD15
def getLUTout(imageMCD15MOD09):
    imageMCD15MOD09 = ee.Image(imageMCD15MOD09);

    return imageMCD15MOD09.select('Lai').min(99).multiply(1).toInt64() \
                        .add(imageMCD15MOD09.select('LaiStdDev').min(99).multiply(1e2).toInt64()) \
                        .add(imageMCD15MOD09.select('Fpar').min(99).multiply(1e4).toInt64()) \
                        .add(imageMCD15MOD09.select('FparStdDev').min(99).multiply(1e6).toInt64()) \
                        .rename('LUTout')

In [67]:
# coded LUT for input landsat image red and nir band
def getLUTL0XinB45(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);
    return imageMCD15.select('SR_B4').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1e3).floor().multiply(1).toInt64() \
                        .add(imageMCD15.select('SR_B5').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1000).floor().multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('VZA').multiply(0.01).round().multiply(1e8).toInt64()) \
                        .add(imageMCD15.select('VAA').multiply(0.01).add(180).round().multiply(1e11).toInt64()) \
                        .add(imageMCD15.select('SZA').multiply(0.01).round().multiply(1e13).toInt64()) \
                        .add(imageMCD15.select('SAA').multiply(0.01).add(180).round().multiply(1e16).toInt64()) \
                        .add(imageMCD15.select('LC_Type3').multiply(1e18).toInt64()) \
                        .rename('LUTL0XinB45')







In [68]:
# coded LUT for input landsat image green and swir
def getLUTL0XinB36(imageMCD15):
    imageMCD15 = ee.Image(imageMCD15);
    return imageMCD15.select('SR_B3').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1e3).floor().multiply(1).toInt64() \
                        .add(imageMCD15.select('SR_B6').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1000).floor().multiply(1e6).toInt64()) \
                        .add(imageMCD15.select('VZA').multiply(0.01).round().multiply(1e8).toInt64()) \
                        .add(imageMCD15.select('VAA').multiply(0.01).add(180).round().multiply(1e11).toInt64()) \
                        .add(imageMCD15.select('SZA').multiply(0.01).round().multiply(1e13).toInt64()) \
                        .add(imageMCD15.select('SAA').multiply(0.01).add(180).round().multiply(1e16).toInt64()) \
                        .add(imageMCD15.select('LC_Type3').multiply(1e18).toInt64()) \
                        .rename('LUTL0XinB36')


In [71]:
# sample worldwide 
# check each year and month
nsamSat = 0
nsamunSat = 0
for year in range(2020,2021):#

        
    # specify land cover projected into MOD15 projection and aggregated using median to outputScale    
    projectionMCD15=  ee.ImageCollection('MODIS/006/MCD15A3H').first().select('Lai').projection()
    projectionMCD12 =  ee.ImageCollection("MODIS/006/MCD12Q1").first().select('LC_Type3').projection()
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate(ee.Date.fromYMD(year,1,1),ee.Date.fromYMD(year,12,31)).first() \
                     .select('LC_Type3') \
                     .reproject(crs= projectionMCD15,crsTransform= None,scale=projectionMCD12.nominalScale() ) \
                     .reduceResolution(reducer= ee.Reducer.median(),maxPixels= 1024,bestEffort=False) \
                     .reproject(crs= projectionMCD15.atScale(outputScale))      

    # sample monthly data
    for month in [6,7,5,8,4,9,3,10,2,11,1,12]:  
        
        #sample WRS path rows
        for path in range(1,233,deltaPath): 
            for row in range(1,248,deltaRow): 
                
                # sample deltaDay intervals of each month
                for daystart in range (1,30-deltaDay,deltaDay):
                    
                    # find Landsat images for paht , row and date interval between 0 and maxCloudCover
                    imageLC = ee.ImageCollection(collectionLandsatL2a).filterMetadata('WRS_PATH','equals',path) \
                                                                          .filterMetadata('WRS_ROW','equals',row) \
                                                                          .filterDate(ee.Date.fromYMD(year,month,daystart),ee.Date.fromYMD(year,month,daystart).advance(deltaDay,'days')) \
                                                                          .filter(ee.Filter.lte('CLOUD_COVER_LAND',cloudCoverL0X))
                    # only proceed if there is at least one filtered landsat image (we will only use the first one found)
                    if (imageLC.size().getInfo() > 0) : 

                        # only sample the first scene we found in the interval of deltaDay's 
                        imageLC = imageLC.first()
                        projectionLC = imageLC.select('QA_PIXEL').projection()
                        
                        # get date range of imageLC and print it out
                        d1 = ee.Date(imageLC.get('system:time_start'))
                        d2 = ee.Date(imageLC.get('system:time_end'))     
                        print('year',year,'month',month,'path',path,'row',row)

                        # get  MOD15 product containing the aquistion of the  Landsat image
                        imageMCD15 = ee.ImageCollection('MODIS/006/MCD15A3H').map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                            .filter(ee.Filter.dateRangeContains('daterange', d1)) 

                        # only proceed if there is a MCD15 product containing the Landsat image aquisition
                        if (imageMCD15.size().getInfo() > 0)  :
                    
                            #randomly sample a sub-image to avoid memory issues, 
                            # add a validPixel we will use to determine aggregation and unmask since we will accept some missing data
                            # imageGRID = imageLC.geometry().coveringGrid(projectionMCD15,subimageSize).randomColumn(columnName='randomIndex',seed=randomSeed)                                
                            imageLC = imageLC.addBands(ee.ImageCollection(collectionLandsatL1c) \
                                                                          .filterMetadata('system:index','equals',imageLC.get('system:index')) \
                                                                          .first() \
                                                                          .select(['VZA','VAA','SZA','SAA'])) \
                                             .clip(ee.Feature(imageGRID.sort('randomIndex').first()).geometry()) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<0).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<1).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<2).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<3).eq(0)) \
                                             .updateMask(imageLC.select('QA_PIXEL').bitwiseAnd(1<<4).eq(0)) 
        
                            
                            # get the fraction of area with unmasked LC data 
                            validArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            totalArea = ee.Number(ee.Image.pixelArea().mask(imageLC.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=30,bestEffort=True).get('area')).getInfo()
                            
                            fracAreaLC = 0 
                            if (totalArea > 0) :
                                fracAreaLC = validArea / totalArea
                            #print('fracAreaLC:',fracAreaLC)  

                            # proceed if there is sufficient landsat area
                            if ( fracAreaLC > minfracAreaLC) :
                                                
                                # get MCD15 image from collection, clip to Landsat subimage
                                imageMCD15 = imageMCD15.first().clip(imageLC.geometry()) 
                                                
                                # update mask and add land cover and a saturation flag band
                                imageMCD15 = imageMCD15.updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                                                .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                                                .addBands( imageMCD12Q1 ).addBands(imageMCD15.select('FparLai_QC').rename(['satFlag']).bitwiseAnd(1<<5).eq(1)) 
                            
                                # get the fraction of area with unmasked MCD15 data 
                                validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),crs=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                fracAreaMCD15 = 0 
                                if (totalArea > 0) :
                                    fracAreaMCD15 = validArea / totalArea
                                #print('fracAreaMCD15:',fracAreaMCD15) 
                           
                                # sample if there is sufficient unmasked area
                                if ( fracAreaMCD15> minfracAreaMCD15 ) :
                                                
                                    
                                    # add a view zenith angle band and a validPixel band to the landsat image
                                    # then unmask and aggregate to output scale, we unmask first to allow for <100% validPixel s
                                    imageLCtoMCD = imageLC.addBands(imageLC.select('QA_PIXEL').gt(-1).rename('validPixel')) \
                                                         .unmask() \
                                                         .reproject(crs= projectionMCD15,scale=projectionLC.nominalScale() ) \
                                                         .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=maxPixels,bestEffort=False) \
                                                         .reproject(crs= projectionMCD15 , scale= outputScale) 
                                    
                                    # mask bands not meeting fractional pixel threshold
                                    # we then have to divide by the fractional validPixel band to adjust for  zero masked values 
                                    validLCtoMCD = imageLCtoMCD.select('validPixel').gte(minfracValidLC);
                                    imageLCtoMCD = imageLCtoMCD.updateMask(validLCtoMCD) \
                                                     .divide(imageLCtoMCD.select('validPixel'))


                                    # add the aggregated LC image to the modis image and update the MCD image with the valid mask
                                    imageMCD15 = imageMCD15.addBands(imageLCtoMCD) \
                                                            .updateMask(validLCtoMCD) 
                                                            
                                    # get the fraction of area with unmasked LC data 
                                    validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('QA_PIXEL').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageMCD15.geometry(),scale=outputScale,bestEffort=True).get('area')).getInfo()
                                    fracAreaLCtoMCD = 0 
                                    if (totalArea > 0) :
                                        fracAreaLCtoMCD = validArea / totalArea
                                    #print('fracAreaLCtoMCD:',fracAreaLCtoMCD)  
                                    
                                    # check if we have enough valid cloud free aggregated matches
                                    if ( fracAreaLCtoMCD > minfracAreaLCtoMCD ) :
 

                                        imageMCD15 = imageMCD15.select('satFlag').addBands(getLUTout(imageMCD15)) \
                                                                                 .addBands(getLUTL0XinB45(imageMCD15)) \
                                                                                 .addBands(getLUTL0XinB36(imageMCD15)) \
                                                                                 .addBands(ee.Image.pixelLonLat())

                                        sampleData = imageMCD15.updateMask(imageMCD15.select('satFlag').lt(ee.Image.constant(0.1))).sample(projection=projectionMCD15.atScale(outputScale), numPixels=maxSample, dropNulls=True)
                                        sampleOutput = sampleData.aggregate_array('LUTout').flatten().getInfo()
                                        sampleInputB45 = sampleData.aggregate_array('LUTL0XinB45').flatten().getInfo()
                                        sampleInputB36 = sampleData.aggregate_array('LUTL0XinB36').flatten().getInfo()
                                        sampleInputLon = sampleData.aggregate_array('Llngitude').flatten().getInfo()
                                        sampleInputLat = sampleData.aggregate_array('latitude').flatten().getInfo()
                            
                                        if (len(sampleOutput)>0) :

                                            nsam = len(sampleOutput)
                                            nsamunSat = nsamunSat + nsam
                                            print('cumulative unsat samples:',nsamunSat,'nsaminput unsat',len(sampleInputB45))
                                            
                                            # write list after each landsat scene is done (comment out if it is too frequent)
                                            with open(unsatInputFileNameLon, 'ab') as fp1:
                                                pickle.dump(sampleInputLon, fp1)
                                            with open(unsatInputFileNameLat, 'ab') as fp1:
                                                pickle.dump(sampleInputLat, fp1)
                                            with open(unsatInputFileNameB36, 'ab') as fp1:
                                                pickle.dump(sampleInputB36, fp1)
                                            with open(unsatInputFileNameB45, 'ab') as fp1:
                                                pickle.dump(sampleInputB45, fp1)
                                            with open(unsatOutputFileName, 'ab') as fp2:
                                                pickle.dump(sampleOutput, fp2)                                        
                                            with open(unsatYearFileName, 'ab') as fp5:
                                                pickle.dump([year]*nsam, fp5)
                                            with open(unsatMonthFileName, 'ab') as fp6:
                                                pickle.dump([month]*nsam, fp6)
                                            with open(unsatDayFileName, 'ab') as fp7:
                                                pickle.dump([daystart]*nsam, fp7)
                                            with open(unsatPathFileName, 'ab') as fp8:
                                                pickle.dump([path]*nsam, fp8)   
                                            with open(unsatRowFileName, 'ab') as fp9:
                                                pickle.dump([row]*nsam, fp9)   

                                        sampleData = imageMCD15.updateMask(imageMCD15.select('satFlag').gt(ee.Image.constant(0.9))).sample(projection=projectionMCD15.atScale(outputScale), numPixels=maxSample, dropNulls=True)
                                        sampleOutput = sampleData.aggregate_array('LUTout').flatten().getInfo()
                                        sampleInputB45 = sampleData.aggregate_array('LUTL0XinB45').flatten().getInfo()
                                        sampleInputB36 = sampleData.aggregate_array('LUTL0XinB36').flatten().getInfo()
                                        sampleInputLon = sampleData.aggregate_array('Llngitude').flatten().getInfo()
                                        sampleInputLat = sampleData.aggregate_array('latitude').flatten().getInfo()
 
                                        if (len(sampleOutput)>0) :

                                            nsam = len(sampleOutput)
                                            nsamSat = nsamSat + nsam
                                            print('cumulative sat samples:',nsamSat,'nsaminput sat',len(sampleInputB45))
                    
                                            # write list after each landsat scene is done (comment out if it is too frequent)
                                            with open(satInputFileNameLon, 'ab') as fp1:
                                                pickle.dump(sampleInputLon, fp1)
                                            with open(satInputFileNameLat, 'ab') as fp1:
                                                pickle.dump(sampleInputLat, fp1)
                                            with open(satInputFileNameB36, 'ab') as fp1:
                                                pickle.dump(sampleInputB36, fp1)
                                            with open(satInputFileNameB45, 'ab') as fp1:
                                                pickle.dump(sampleInputB45, fp1)
                                            with open(satOutputFileName, 'ab') as fp2:
                                                pickle.dump(sampleOutput, fp2)                                        
                                            with open(satYearFileName, 'ab') as fp5:
                                                pickle.dump([year]*nsam, fp5)
                                            with open(satMonthFileName, 'ab') as fp6:
                                                pickle.dump([month]*nsam, fp6)
                                            with open(satDayFileName, 'ab') as fp7:
                                                pickle.dump([daystart]*nsam, fp7)
                                            with open(satPathFileName, 'ab') as fp8:
                                                pickle.dump([path]*nsam, fp8)   
                                            with open(satRowFileName, 'ab') as fp9:
                                                pickle.dump([row]*nsam, fp9)  
        


year 2020 month 6 path 1 row 81
year 2020 month 6 path 21 row 1
year 2020 month 6 path 31 row 31
year 2020 month 6 path 61 row 1
year 2020 month 6 path 61 row 11
year 2020 month 6 path 71 row 11
year 2020 month 6 path 91 row 81
cumulative unsat samples: 1244 nsaminput unsat 1244
year 2020 month 6 path 101 row 11
year 2020 month 6 path 111 row 71
year 2020 month 6 path 121 row 31
year 2020 month 6 path 131 row 11
year 2020 month 6 path 131 row 31
year 2020 month 6 path 141 row 21
year 2020 month 6 path 151 row 41
year 2020 month 6 path 161 row 21
year 2020 month 6 path 161 row 31
year 2020 month 6 path 161 row 41
year 2020 month 6 path 161 row 71
year 2020 month 6 path 181 row 51
year 2020 month 6 path 181 row 71
year 2020 month 6 path 191 row 31
year 2020 month 6 path 191 row 41
year 2020 month 6 path 201 row 41
year 2020 month 6 path 221 row 71
year 2020 month 6 path 221 row 81
year 2020 month 6 path 231 row 11
year 2020 month 7 path 31 row 41
year 2020 month 7 path 41 row 31
year 202

KeyboardInterrupt: 

In [42]:
  # check the sampling worked
with open('/home/richard_fernandes/unsatLUToutputLC0Xrun2020', 'rb') as fp1:
    x = pickle.load(fp1)
    print(x)

    LUTout
0  4490214
1  5650422
2   540014
3  5610317
4  5490212
5  4530216
6  4600319


In [72]:
  # check the sampling worked
with open('/home/richard_fernandes/unsatLUTinputLC0XB45run2020', 'rb') as fp1:
    x = pickle.load(fp1)
    print(x)

[3.13062100095e+18, 3.1306210010330004e+18, 3.130621001037e+18, 3.130621001023e+18, 6.130621000993e+18, 6.130621000966001e+18, 6.130621000944e+18, 6.130621000874e+18, 6.130620900918001e+18, 6.130620500868e+18, 6.130619700904001e+18, 6.130619700932001e+18, 6.130619700932e+18, 6.13061970096e+18, 3.130619700991e+18, 3.1306197009930004e+18, 3.130619701038e+18, 3.13061970102e+18, 3.1306197010030003e+18, 3.130619700957e+18, 3.1306198011070003e+18, 3.1306198010600003e+18, 3.1306198010560005e+18, 3.1306198010480005e+18, 3.130619801071e+18, 3.1306198010670003e+18, 3.1306198010510003e+18, 3.130619801068e+18, 3.1306198010600003e+18, 3.130619801029e+18, 6.130619801001e+18, 3.130619801047e+18, 3.1306198011040005e+18, 3.130619901134e+18, 3.1306201011210004e+18, 3.130620501084e+18, 3.1306207010720005e+18, 3.130621001029e+18, 3.1306210010120003e+18, 3.1306210010160005e+18, 6.130621001001e+18, 6.130621000971e+18, 6.130621000947e+18, 6.130621000843e+18, 6.130620700919e+18, 6.130619700968001e+18, 3.13061